In [2]:
import numpy as np
import pyro
import pyro.distributions as dist
import torch

In [3]:
# write a simple model
def model(obs=None, n_obs=None):
    if n_obs == None:
        n_obs = obs.shape[0]
    mu = pyro.sample('mu', dist.Normal(torch.zeros(()), torch.ones(())))
    mean = mu * torch.ones(n_obs,)
    sigma = torch.ones(n_obs,)
    with pyro.plate('data_plate', n_obs):
        data = pyro.sample('data', dist.Normal(mean, sigma), obs=obs)
    return mu, data

def predict(mcmc_samples, n_obs):
    mu = np.array(mcmc_samples['mu'])
    assert mu.shape[0] == n_obs
    return mu

def r_squared(mu, test):
    mean = np.mean(mu, axis=0)
    pearson_corr = np.corrcoef(test, mean * np.ones(len(test),))
    r = pearson_corr[0, 1]
    return np.power(r, 2)

# function to compute coverage
def coverage(mu, obs, hi, lo):
    assert hi > lo
    # generate synthetic samples from normal distribution with mean mu
    m = mu.shape[0]
    # generate synthetic samples for each observation
    # TODO: Figure out how to get correct variance in here
    synth = mu + np.random.normal(loc=0, scale=1, size=(m,))
    # sort synthetic samples for each observation
    sorted_synth = np.sort(synth, axis=0)
    # compute hi and lo index
    lo_idx = int(np.ceil(lo * m))
    hi_idx = int(np.floor(hi * m))
    # get synthetic samples at hi and lo indices
    lo_bound = sorted_synth[lo_idx]
    hi_bound = sorted_synth[hi_idx]
    # is obs in [hi, lo]?
    frac = np.sum(np.logical_and(lo_bound < obs, obs < hi_bound) / (1.0 * len(obs)))
    return frac

In [4]:
# sample from the model: train, test
true_mu, obs = model(n_obs=1000)
obs_train = obs[0:800]
obs_test = obs[800:].numpy()

In [5]:
true_mu

tensor(-0.4049)

In [6]:
# fit the model
pyro.clear_param_store()
kernel = pyro.infer.mcmc.NUTS(model, jit_compile=True)
mcmc = pyro.infer.MCMC(kernel, num_samples=50000, warmup_steps=500)
mcmc.run(obs=obs_train)
mcmc_samples = {k: v.detach().cpu().numpy() for k, v in mcmc.get_samples().items()}
# to start, just compute mu and compare with true_mu

Warmup:   0%|                                           | 0/50500 [00:00, ?it/s]/Users/michelameister/opt/anaconda3/lib/python3.8/site-packages/pyro/poutine/subsample_messenger.py:63: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  result = torch.tensor(0.0, device=self.device)
Sample: 100%|█| 50500/50500 [00:52, 953.00it/s, step size=1.14e+00, acc. prob=0.


In [8]:
n_obs = 50000
mu = predict(mcmc_samples, n_obs)
#r_sq = r_squared(mu, obs_test)
frac = coverage(mu, obs_test, .95, .05)

In [ ]:
print('frac: ' + str(frac))

In [7]:
first_mcmc = {'mu': mcmc_samples['mu'][:500]}
mu = predict(first_mcmc, 500)
#r_sq = r_squared(mu, obs_test)
frac = coverage(mu, obs_test, .95, .05)
print('frac: ' + str(frac))

frac: 0.9199999999999999


In [ ]:
mean = np.mean(mu, axis=0)
pearson_corr = np.corrcoef(obs_test, mean * np.ones(len(obs_test),))
r = pearson_corr[0, 1]
print(r)

In [ ]:
mean = np.mean(mu, axis=0)
print(mean)
pearson_corr = np.corrcoef(obs_test, mean * np.ones(len(obs_test),))
print(pearson_corr)

In [ ]:
arr1 = np.array([1, 2, 3])
arr2 = np.array([4, 5, 6])
np.corrcoef(arr1, arr2)

In [ ]:
barr1 = 2 * np.array(np.ones(3,))
barr2 = np.array(np.ones(3,))
np.corrcoef(arr1, arr2)

In [ ]:
barr1.shape

In [ ]:
print('r_sq: ' + str(r_sq))
print('frac: ' + str(frac))

In [ ]:
mu = np.array(mcmc_samples['mu'])
mu.shape

In [ ]:
np.mean(mu, axis=0)

In [ ]:
mean = np.mean(mu, axis=0)
np.ones(50000,)